In [1]:
import numpy as np
import pandas as pd
from scipy.interpolate import UnivariateSpline
from scipy.integrate import trapz, cumtrapz
import optimum_reparamN2 as orN2
import optimum_reparam_N as orN
from joblib import Parallel, delayed


df = pd.read_csv("./data/sample_data2.csv")
print(df.shape)
df.head()

(288, 102)


,C_0,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,...,C_92,C_93,C_94,C_95,C_96,C_97,C_98,C_99,C_100,C_101
0,17.85,18.48,18.80,17.85,17.85,18.48,18.17,17.53,18.17,18.17,...,17.53,18.48,19.12,17.85,16.58,16.26,17.21,16.26,15.3,17.85
1,17.85,18.17,18.48,17.53,17.53,18.48,18.17,17.53,18.17,18.17,...,17.53,18.17,19.12,17.85,16.58,15.94,17.21,16.26,15.3,17.85
2,17.85,18.17,18.48,17.53,17.85,18.48,18.17,17.53,18.17,18.17,...,17.53,18.17,19.12,17.85,16.58,16.26,17.21,16.26,15.3,17.85
3,17.85,18.17,18.80,17.53,17.53,18.48,18.17,17.53,18.17,18.17,...,17.53,18.17,19.12,17.85,16.58,15.94,17.21,16.26,15.3,17.85
4,17.85,18.17,18.80,17.53,17.85,18.48,18.17,17.53,18.17,18.17,...,17.53,18.17,19.12,17.85,16.58,16.26,17.21,16.26,15.3,17.85


In [2]:
start = 0
n_timepts = 20

In [3]:
def restart():    
    F = df[start:n_timepts].iloc[:-1,:-1].to_numpy()
    time_points = np.linspace(0, 1, F.shape[0])

    return F, time_points

F, time_points = restart()

In [4]:
global g 
global gam

def f_to_srsf(f, time, smooth=False):
    eps = np.finfo(np.double).eps
    f0, g, g2 = gradient_spline(time, f, smooth)
    q = g / np.sqrt(np.fabs(g) + eps)
    return q

def gradient_spline(time, f, smooth=False):
    M = f.shape[0]

    if f.ndim > 1:
        N = f.shape[1]
        f0 = np.zeros((M, N))
        g = np.zeros((M, N))
        g2 = np.zeros((M, N))
        for k in range(0, N):
            if smooth:
                spar = time.shape[0] * (.025 * np.fabs(f[:, k]).max()) ** 2
            else:
                spar = 0
            tmp_spline = UnivariateSpline(time, f[:, k], s=spar)
            f0[:, k] = tmp_spline(time)
            g[:, k] = tmp_spline(time, 1)
            g2[:, k] = tmp_spline(time, 2)
    else:
        if smooth:
            spar = time.shape[0] * (.025 * np.fabs(f).max()) ** 2
        else:
            spar = 0
        tmp_spline = UnivariateSpline(time, f, s=spar)
        f0 = tmp_spline(time)
        g = tmp_spline(time, 1)
        g2 = tmp_spline(time, 2)

    return f0, g, g2

def optimum_reparam(q1, time, q2, method="DP2", lam=0.0, penalty="roughness", grid_dim=7):
    if penalty == "l2gam" and (method == "DP" or method == "DP2"):
        raise Exception('penalty not implemented')
    if penalty == "l2psi" and (method == "DP" or method == "DP2"):
        raise Exception('penalty not implemented')
    if penalty == "geodesic" and (method == "DP" or method == "DP2"):
        raise Exception('penalty not implemented')
    
    if method == "DP2":
        if q1.ndim == 1 and q2.ndim == 1:
            gam = orN2.coptimum_reparam(np.ascontiguousarray(q1), time,
                                        np.ascontiguousarray(q2), lam, grid_dim)

        if q1.ndim == 1 and q2.ndim == 2:
            gam = orN2.coptimum_reparamN(np.ascontiguousarray(q1), time,
                                        np.ascontiguousarray(q2), lam, grid_dim)

        if q1.ndim == 2 and q2.ndim == 2:
            gam = orN2.coptimum_reparamN2(np.ascontiguousarray(q1), time,
                                        np.ascontiguousarray(q2), lam, grid_dim)
        
    else:
        raise Exception('Invalid Optimization Method')

    return gam

def elastic_distance(f1, f2, time, method="DP2", lam=0.0):
    q1 = f_to_srsf(f1, time)
    q2 = f_to_srsf(f2, time)

    gam = optimum_reparam(q1, time, q2, method, lam)
    fw = warp_f_gamma(time, f2, gam)
    qw = warp_q_gamma(time, q2, gam)

    Dy = np.sqrt(trapz((qw - q1) ** 2, time))
    M = time.shape[0]

    time1 = np.linspace(0,1,M)
    binsize = np.mean(np.diff(time1))
    psi = np.sqrt(np.gradient(gam,binsize))
    q1dotq2 = trapz(psi, time1)
    if q1dotq2 > 1:
        q1dotq2 = 1
    elif q1dotq2 < -1:
        q1dotq2 = -1

    Dx = np.real(np.arccos(q1dotq2))

    return Dy, Dx

def elastic_outliers(F, depths):

    amp = depths['amplitude']
    phs = depths['phase']

    amp_100 = np.max(amp)
    phs_100 = np.max(phs)

    amp_50 = np.percentile(amp, 50)
    phs_50 = np.percentile(phs, 50)

    amp_iqr = amp_100 - amp_50 
    phs_iqr = phs_100 - phs_50

    amp_lim = max(amp_50 - 1.5 * amp_iqr, 0)
    phs_lim = max(phs_50 - 1.5 * phs_iqr, 0)

    amp_thre = np.percentile(amp, 0.5 * 100)
    phs_thre = np.percentile(phs, 0.5 * 100)

    amp_out = (amp < amp_lim) & (amp < amp_thre)
    phs_out = (phs < phs_lim) & (phs < phs_thre)

    labels = {'amp': amp_out, 'phs': phs_out}

    return labels

def warp_f_gamma(time, f, gam):
    f_temp = np.interp((time[-1] - time[0]) * gam + time[0], time, f)
    return f_temp

def warp_q_gamma(time, q, gam):
    M = gam.size
    gam_dev = np.gradient(gam, 1 / np.double(M - 1))
    tmp = np.interp((time[-1] - time[0]) * gam + time[0], time, q)

    q_temp = tmp * np.sqrt(gam_dev)

    return q_temp

def distmat(f, f1, time, idx, method):
    N = f.shape[1]
    dp = np.zeros(N)
    da = np.zeros(N)

    for jj in range(N):
        Dy,Dx = elastic_distance(f[:,jj], f1, time, method)

        da[jj] = Dy
        dp[jj] = Dx
    
    return(da, dp)

def elastic_depth(F, time_points, method="DP2", lam=0.0, parallel=True):
    obs, fns = F.shape

    amp_dist = np.zeros((fns,fns))
    phs_dist = np.zeros((fns,fns))

    if parallel:
        out = Parallel(n_jobs=-1)(delayed(distmat)(F, F[:, n], time_points, n, method) for n in range(fns))
        for i in range(0, fns):
            amp_dist[i, :] = out[i][0]
            phs_dist[i, :] = out[i][1]
    else:
        for i in range(0, fns):
            amp_dist[i, :], phs_dist[i, :] = distmat(F, F[:, i], time_points, i, method)
    
    amp_dist = amp_dist + amp_dist.T
    phs_dist = phs_dist + phs_dist.T

    amp = 1 / (1 + np.median(amp_dist,axis=0))
    phase = 1 / (1 + np.median(phs_dist,axis=0))
    phase = ((2+np.pi)/np.pi) * (phase - 2/(2+np.pi))

    return amp, phase

In [5]:
F, time_points = restart()
amp_depth, phs_depth = elastic_depth(F, time_points)
depths = {'amplitude': amp_depth, 'phase': phs_depth}
amp_depth, phs_depth

(array([0.24455954, 0.23574058, 0.21988801, 0.26540288, 0.20919682,
        0.23999841, 0.19681455, 0.21178533, 0.22130809, 0.27043148,
        0.25378555, 0.28276621, 0.22010679, 0.24816159, 0.23620569,
        0.23118673, 0.26257848, 0.22190876, 0.22865631, 0.22022211,
        0.23426764, 0.26368799, 0.23722823, 0.23426764, 0.22000883,
        0.2477153 , 0.24647102, 0.21823807, 0.24455954, 0.2300494 ,
        0.23481698, 0.25912019, 0.23410827, 0.23368597, 0.22784171,
        0.20021857, 0.25912019, 0.24647102, 0.23368597, 0.25872056,
        0.20555252, 0.26293163, 0.24634953, 0.22260249, 0.24905818,
        0.2294437 , 0.25890839, 0.26524014, 0.24905818, 0.27148481,
        0.25808486, 0.27208113, 0.24422319, 0.26760063, 0.24422319,
        0.25089876, 0.26197828, 0.25703677, 0.28192995, 0.26369772,
        0.23387981, 0.2217911 , 0.25808486, 0.25293528, 0.25808486,
        0.2598769 , 0.25728226, 0.27549033, 0.25808486, 0.27645253,
        0.27345249, 0.25808486, 0.2317627 , 0.25

In [6]:
F.shape # (time points, functions)

(19, 101)

In [7]:
len(depths['amplitude'])

101

In [8]:
elastic_out = elastic_outliers(F, depths)
print('number of labels:',len(elastic_out['amp']))
elastic_out['amp']

number of labels: 101


array([False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False])

In [9]:
def prog_update(F, f, depths, labels, method="DP2", lam=0.0, parallel=True):
    obs, fns = F.shape

    amp_dist = np.zeros(fns)
    phs_dist = np.zeros(fns)

    # computing elastic distances 
    amp_dist, phs_dist = distmat(F, f, time_points, 0, method)

    amp_new = 1 / (1 + np.median(amp_dist,axis=0))
    phase_new = 1 / (1 + np.median(phs_dist,axis=0))
    phase_new = ((2+np.pi)/np.pi) * (phase_new - 2/(2+np.pi))

    # computing outlier label for new amp/phase depth
    amp_depths = depths['amplitude']
    phs_depths = depths['phase']

    amp_100 = np.max(amp_depths)
    phs_100 = np.max(phs_depths)

    amp_50 = np.percentile(amp_depths, 50)
    phs_50 = np.percentile(phs_depths, 50)

    amp_iqr = amp_100 - amp_50 
    phs_iqr = phs_100 - phs_50

    amp_lim = max(amp_50 - 1.5 * amp_iqr, 0)
    phs_lim = max(phs_50 - 1.5 * phs_iqr, 0)

    amp_thre = np.percentile(amp_depths, 0.5 * 100)
    phs_thre = np.percentile(phs_depths, 0.5 * 100)

    amp_out = (amp_new < amp_lim) & (amp_new < amp_thre)
    phs_out = (phase_new < phs_lim) & (phase_new < phs_thre)

    # updating labels
    labels['amp'] = np.append(labels['amp'], amp_out)
    labels['phs'] = np.append(labels['phs'], phs_out)

    # updating depths
    depths['amplitude'] = np.append(depths['amplitude'], amp_new)
    depths['phase'] = np.append(depths['phase'], phase_new)

    return labels

In [10]:
F_new = df[start:n_timepts].iloc[:-1,101].T.to_numpy()
out_labels = prog_update(F, F_new, depths, elastic_out)

In [11]:
len(depths['amplitude'])

102

In [12]:
print('number of labels:',len(out_labels['amp']))
out_labels['amp']

number of labels: 102


array([False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

In [13]:
df.iloc[-1,:]

C_0      16.89
C_1      16.89
C_2      17.53
C_3      16.58
C_4      16.58
         ...  
C_97     15.94
C_98     17.21
C_99     16.26
C_100    15.30
C_101    17.85
Name: 287, Length: 102, dtype: float64

In [14]:
x = df.iloc[-1,:].to_numpy().reshape(1, -1) # new time points shape (1, 102)
x

array([[16.89, 16.89, 17.53, 16.58, 16.58, 17.21, 16.58, 16.26, 17.21,
        16.89, 17.85, 15.62, 16.89, 17.85, 17.85, 16.89, 16.89, 16.58,
        17.53, 15.94, 17.21, 17.53, 17.53, 16.89, 16.58, 16.89, 17.21,
        16.58, 17.21, 18.17, 17.85, 16.58, 15.94, 17.21, 17.21, 16.89,
        17.85, 18.17, 17.85, 17.21, 15.94, 16.89, 16.89, 15.94, 15.62,
        16.89, 16.89, 16.26, 16.89, 17.21, 17.85, 17.21, 15.94, 16.58,
        16.58, 15.62, 16.26, 17.53, 17.53, 16.89, 16.58, 17.53, 17.53,
        16.89, 16.58, 16.89, 17.85, 15.94, 16.89, 17.21, 17.21, 16.58,
        16.89, 17.53, 17.21, 16.26, 15.62, 16.58, 16.26, 15.3 , 17.21,
        17.85, 18.17, 16.89, 15.62, 16.58, 16.26, 15.62, 16.26, 16.89,
        17.53, 15.94, 16.58, 16.89, 17.85, 16.58, 16.58, 15.94, 17.21,
        16.26, 15.3 , 17.85]])

In [15]:
x.shape

(1, 102)

In [16]:
F = df[start:n_timepts].iloc[:-1,:].to_numpy()
time_points = np.linspace(0, 1, F.shape[0])

In [17]:
x[:,0]

array([16.89])

In [18]:
np.append(F[:,0], x[:,0])

array([17.85, 17.85, 17.85, 17.85, 17.85, 17.85, 17.85, 17.85, 17.85,
       17.85, 17.85, 16.89, 16.89, 16.89, 16.89, 16.89, 16.89, 16.89,
       18.8 , 16.89])

In [19]:
np.vstack((F, x))

array([[17.85, 18.48, 18.8 , ..., 16.26, 15.3 , 17.85],
       [17.85, 18.17, 18.48, ..., 16.26, 15.3 , 17.85],
       [17.85, 18.17, 18.48, ..., 16.26, 15.3 , 17.85],
       ...,
       [16.89, 16.89, 17.21, ..., 16.26, 15.3 , 17.85],
       [18.8 , 18.48, 17.21, ..., 16.26, 15.3 , 17.85],
       [16.89, 16.89, 17.53, ..., 16.26, 15.3 , 17.85]])

In [19]:
def inc_update(F, x, depths, threshold=0.8, method="DP2", lam=0.0, parallel=True):
    F_inc = np.vstack((F, x)) 
    obs, fns = F.shape
    obs2, fns2 = F_inc.shape
    
    if (fns != fns2):
        raise Exception('Error: number of functions in increment != number of functions in F')
    
    t1 = np.linspace(0, 1, F.shape[0]) # (19, 102)
    t2 = np.linspace(0, 1, F_inc.shape[0]) # (20, 102)

    # iterating through the functions
    for i in range(fns):
        f1 = F[:,i]
        f2 = F_inc[:,i]

        q1 = f_to_srsf(f1, t1) # previous without new point
        q2 = f_to_srsf(f2, t2) # with new point

        gamma = optimum_reparam(q1, t2, q2, method, lam)
        q1_aligned = np.interp(t2, np.linspace(0, 1, len(q1)), q1)

        q_diff = np.linalg.norm(q2-q1_aligned)
        print(q_diff)

        # Recompute depth if change is significant
        if q_diff > threshold:
            amp_new, phase_new = distmat(f2, method)  # EDIT FUNCTION PARAMS
            depths['amplitude'] = np.append(depths['amplitude'], amp_new) #NEED TO CHANGE DEPTHS[i] at specific point
            depths['phase'] = np.append(depths['phase'], phase_new)  
        return depths[i]
    return depths

In [25]:
df.shape

In [20]:
out_labels = inc_update(F, x, depths)

20.928557013084987
19.59511141671914
8.471094571280535
4.5233508029951315
22.581078037395624
20.235959455099255
26.628676843995475
22.47342553730497
23.305269408077287
13.056393551253402
5.801776034065769
4.9672471696461376
20.824937745885027
17.508900198461742
17.58144533705468
19.552105396367466
17.349054895938366
21.113507437000337
21.328810388295306
19.743604374033858
22.539078134702713
15.406140094941724
19.5862979997338
22.539078134702685
24.585093490585567
17.577697713775137
19.899300402437184
21.98559302836591
20.928557013085
19.263709878679354
19.410478008139155
19.186671781398747
21.213928591423578
21.101172065505292
22.847289412119363
22.84598502048254
19.18667178139874
19.384198132412525
21.101172065505267
19.192799504962046
24.245817171226033
15.484301771811497
17.798176720538308
24.206438322400817
10.183519894165261
10.463561011152967
9.929207497216122
12.067461992968187
9.93999658830122
8.965089324124557
8.823264365075437
9.368412674698043
13.37562340013957
9.38672388993